In [12]:
import numpy as np

# WRF Python Translation

- [wrf.f90](#wrf.f90)
    - [module_top_wrf.f90](#module_top_wrf.f90) (only wrf_init, wrf_dfi, wrf_run, wrf_finalize)
        - [module_driver_constants](#module_driver_constants)
            - `module_driver_constants()`
        - [module_machine](#module_machine)
            - [rlocproc](#rlocproc) `rlocproc(p, maxi, nproc, ml, mr, ret)`
            - `locproc(i, m, numpart)`
            - `patchmap(res, y, x, py, px)`
            - `region_bounds(region_start, region_end, num_p, p, patch_start, patch_end)`
            - `least_aspect(nparts, minparts_y, minparts_x, nparts_y, nparts_x)`
            - `init_module_machine()`
            - `wrf_sizeof_integer(retval)`
            - `wrf_sizeof_real(retval)`
            - `wrf_sizeof_doubleprecision(retval)`
            - `wrf_sizeof_logical(retval)`
        - module_domain
        - module_integrate
        - module_configure
        - module_check_a_mundo
        - module_timing
        - module_wrf_error
        - module_nesting
        - module_dm, ONLY : wrf_dm_initialize,wrf_get_hostid,domain_active_this_task,mpi_comm_allcompute
        - module_cpl, ONLY : coupler_on, cpl_finalize, cpl_defdomain

---

# `wrf.f90`

## Modules used:

- module_wrf_top (only wrf_init, wrf_dfi, wrf_run, wrf_finalize)

## Full Code:

```{FORTRAN}
PROGRAM wrf

   USE module_wrf_top, only : wrf_init, wrf_dfi, wrf_run, wrf_finalize

   IMPLICIT NONE

  CALL wrf_init

  CALL wrf_dfi

  CALL wrf_run

  CALL wrf_finalize

END PROGRAM wrf
```

## Python Translation:

### `wrf()`

In [1]:
def wrf():
    # module_wrf_top
    wrf_init()
    # module_wrf_top
    wrf_dfi()
    # module_wrf_top
    wrf_run()
    # module_wrf_top
    wrf_finalize()

---

# `module_top_wrf.f90`

ONLY:
 - wrf_init
 - wrf_dfi
 - wrf_run
 - wrf_finalize

## Modules used:

- module_machine
- module_domain
- module_integrate
- module_driver_constants
- module_configure
- module_check_a_mundo
- module_timing
- module_wrf_error
- module_nesting
- module_dm (only wrf_dm_initialize,wrf_get_hostid,domain_active_this_task,mpi_comm_allcompute)
- module_cpl (only coupler_on, cpl_finalize, cpl_defdomain)

## Full Code

```{FORTRAN}
MODULE module_wrf_top

   USE module_machine
   USE module_domain
   USE module_integrate
   USE module_driver_constants
   USE module_configure
   USE module_check_a_mundo
   USE module_timing
   USE module_wrf_error
   USE module_nesting
   USE module_dm, ONLY : wrf_dm_initialize,wrf_get_hostid,domain_active_this_task,mpi_comm_allcompute
   USE module_cpl, ONLY : coupler_on, cpl_finalize, cpl_defdomain

   IMPLICIT NONE

   REAL    :: time
   INTEGER :: loop , &
              levels_to_process
   TYPE (domain) , POINTER :: keep_grid, grid_ptr, null_domain
   TYPE (domain) , pointer :: parent_grid, new_nest
   LOGICAL                                :: a_nest_was_opened
   TYPE (grid_config_rec_type), SAVE :: config_flags
   INTEGER        :: kid, nestid
   INTEGER                 :: number_at_same_level
   INTEGER                 :: time_step_begin_restart
   INTEGER :: max_dom , domain_id , fid , oid , idum1 , idum2 , ierr
   INTEGER :: debug_level
   LOGICAL :: input_from_file
   INTEGER                 :: nbytes
   INTEGER, PARAMETER      :: configbuflen = 4* 65536
   INTEGER                 :: configbuf( configbuflen )
   LOGICAL , EXTERNAL      :: wrf_dm_on_monitor
   CHARACTER (LEN=256)     :: rstname
   CHARACTER (LEN=80)      :: message
   CHARACTER (LEN=256) , PRIVATE :: a_message
   INTERFACE 
     SUBROUTINE Setup_Timekeeping( grid )
      USE module_domain
      TYPE(domain), POINTER :: grid
     END SUBROUTINE Setup_Timekeeping

     SUBROUTINE wrf_dfi_write_initialized_state( )
     END SUBROUTINE wrf_dfi_write_initialized_state
 
     SUBROUTINE wrf_dfi_startfwd_init( )
     END SUBROUTINE wrf_dfi_startfwd_init
     
     SUBROUTINE wrf_dfi_startbck_init( )
     END SUBROUTINE wrf_dfi_startbck_init
     
     SUBROUTINE wrf_dfi_bck_init( )
     END SUBROUTINE wrf_dfi_bck_init
     
     SUBROUTINE wrf_dfi_fwd_init( )
     END SUBROUTINE wrf_dfi_fwd_init
     
     SUBROUTINE wrf_dfi_fst_init( )
     END SUBROUTINE wrf_dfi_fst_init
     
     SUBROUTINE wrf_dfi_array_reset ( )
     END SUBROUTINE wrf_dfi_array_reset

     SUBROUTINE med_nest_initial ( parent , grid , config_flags )
       USE module_domain
       USE module_configure
       TYPE (domain), POINTER ::  grid , parent
       TYPE (grid_config_rec_type) config_flags
     END SUBROUTINE med_nest_initial

   END INTERFACE

CONTAINS

   SUBROUTINE wrf_init( no_init1 )

     LOGICAL, OPTIONAL, INTENT(IN) :: no_init1
     INTEGER i, myproc, nproc, hostid, loccomm, ierr, buddcounter, mydevice, save_comm
     INTEGER, ALLOCATABLE :: hostids(:), budds(:)
     CHARACTER*512 hostname
     CHARACTER*512 mminlu_loc
   CHARACTER (LEN=10) :: release_version = 'V3.8.1    '

   program_name = "WRF " // TRIM(release_version) // " MODEL"

   CALL init_modules(1)
   IF ( .NOT. PRESENT( no_init1 ) ) THEN
     
     CALL WRFU_Initialize( defaultCalKind=WRFU_CAL_GREGORIAN )
   ENDIF
   
   CALL init_modules(2)

   CALL wrf_get_dm_communicator( save_comm )
   CALL wrf_set_dm_communicator( mpi_comm_allcompute )
   IF ( wrf_dm_on_monitor() ) THEN
     CALL initial_config
   ENDIF
   CALL get_config_as_buffer( configbuf, configbuflen, nbytes )
   CALL wrf_dm_bcast_bytes( configbuf, nbytes )
   CALL set_config_as_buffer( configbuf, configbuflen )
   CALL wrf_dm_initialize
   CALL wrf_set_dm_communicator( save_comm )
   CALL set_derived_rconfigs
   CALL check_nml_consistency
   CALL set_physics_rconfigs
   CALL nl_get_debug_level ( 1, debug_level )
   CALL set_wrf_debug_level ( debug_level )

   NULLIFY( null_domain )

   CALL nl_get_max_dom( 1, max_dom )
   IF ( max_dom > 1 ) THEN
   END IF

   CALL       wrf_message ( program_name )
   CALL       wrf_debug ( 100 , 'wrf: calling alloc_and_configure_domain ' )
   CALL alloc_and_configure_domain ( domain_id  = 1 ,                  &
                               active_this_task = domain_active_this_task(1), &
                                     grid       = head_grid ,          &
                                     parent     = null_domain ,        &
                                     kid        = -1                   )
   CALL       wrf_debug ( 100 , 'wrf: calling model_to_grid_config_rec ' )
   CALL model_to_grid_config_rec ( head_grid%id , model_config_rec , config_flags )
   CALL       wrf_debug ( 100 , 'wrf: calling set_scalar_indices_from_config ' )
   CALL set_scalar_indices_from_config ( head_grid%id , idum1, idum2 )
   CALL       wrf_debug ( 100 , 'wrf: calling init_wrfio' )
   CALL init_wrfio

   CALL wrf_get_dm_communicator( save_comm )
   CALL wrf_set_dm_communicator( mpi_comm_allcompute )
   CALL get_config_as_buffer( configbuf, configbuflen, nbytes )
   CALL wrf_dm_bcast_bytes( configbuf, nbytes )
   CALL set_config_as_buffer( configbuf, configbuflen )
   CALL wrf_set_dm_communicator( save_comm )

   IF ( head_grid%dfi_opt .NE. DFI_NODFI ) head_grid%dfi_stage = DFI_SETUP

   CALL Setup_Timekeeping (head_grid)

 IF ( domain_active_this_task(1) ) THEN
   CALL med_initialdata_input( head_grid , config_flags )

   IF ( config_flags%write_restart_at_0h ) THEN
      CALL med_restart_out ( head_grid, config_flags )

      CALL wrf_debug ( 0 , ' 0 h restart only wrf: SUCCESS COMPLETE WRF' )

      CALL wrf_finalize( )
   END IF
  ENDIF  

   head_grid%start_subtime = domain_get_start_time ( head_grid )
   head_grid%stop_subtime = domain_get_stop_time ( head_grid )

 IF ( domain_active_this_task(1) ) THEN
 
   IF ( head_grid%dfi_opt .NE. DFI_NODFI ) THEN
      CALL alloc_doms_for_dfi ( head_grid )
   END IF

   IF (coupler_on) CALL cpl_defdomain( head_grid ) 
  ENDIF  

   END SUBROUTINE wrf_init

   SUBROUTINE wrf_run( )

   CALL       wrf_debug ( 100 , 'wrf: calling integrate' )
   CALL integrate ( head_grid )
   CALL       wrf_debug ( 100 , 'wrf: back from integrate' )

   END SUBROUTINE wrf_run

   SUBROUTINE wrf_finalize( no_shutdown )

     LOGICAL, OPTIONAL, INTENT(IN) :: no_shutdown
 
   CALL med_shutdown_io ( head_grid , config_flags )
   CALL       wrf_debug ( 100 , 'wrf: back from med_shutdown_io' )
   CALL       wrf_debug (   0 , 'wrf: SUCCESS COMPLETE WRF' )

   IF ( .NOT. PRESENT( no_shutdown ) ) THEN
     
      IF (coupler_on) THEN 
         CALL cpl_finalize() 
      ELSE
         CALL WRFU_Finalize
         CALL wrf_shutdown
      ENDIF
   ENDIF

   END SUBROUTINE wrf_finalize

   SUBROUTINE wrf_dfi()

      IMPLICIT NONE

      IF ( config_flags%dfi_opt .NE. DFI_NODFI ) THEN
   
         SELECT CASE ( config_flags%dfi_opt ) 
     
            CASE (DFI_DFL)
               wrf_err_message = 'Initializing with DFL'
               CALL wrf_message(TRIM(wrf_err_message))
   
               wrf_err_message = '   Filtering forward in time'
               CALL wrf_message(TRIM(wrf_err_message))
   
               CALL wrf_dfi_fwd_init()
               CALL wrf_run()
   
               CALL wrf_dfi_array_reset()
   
               CALL wrf_dfi_fst_init()
   
               IF ( config_flags%dfi_write_filtered_input ) THEN
                  CALL wrf_dfi_write_initialized_state()
               END IF
   
            CASE (DFI_DDFI)
               wrf_err_message = 'Initializing with DDFI'
               CALL wrf_message(TRIM(wrf_err_message))
   
               wrf_err_message = '   Integrating backward in time'
               CALL wrf_message(TRIM(wrf_err_message))
   
               CALL wrf_dfi_bck_init()
               CALL wrf_run()
   
               wrf_err_message = '   Filtering forward in time'
               CALL wrf_message(TRIM(wrf_err_message))
   
               CALL wrf_dfi_fwd_init()
               CALL wrf_run()
   
               CALL wrf_dfi_array_reset()
   
               CALL wrf_dfi_fst_init()
   
               IF ( config_flags%dfi_write_filtered_input ) THEN
                  CALL wrf_dfi_write_initialized_state()
               END IF
   
            CASE (DFI_TDFI)
               wrf_err_message = 'Initializing with TDFI'
               CALL wrf_message(TRIM(wrf_err_message))
   
               wrf_err_message = '   Integrating backward in time'
               CALL wrf_message(TRIM(wrf_err_message))
   
               CALL wrf_dfi_bck_init()
               CALL wrf_run()
   
               CALL wrf_dfi_array_reset()
   
               wrf_err_message = '   Filtering forward in time'
               CALL wrf_message(TRIM(wrf_err_message))
   
               CALL wrf_dfi_fwd_init()
               CALL wrf_run()
   
               CALL wrf_dfi_array_reset()
   
               CALL wrf_dfi_fst_init()
   
               IF ( config_flags%dfi_write_filtered_input ) THEN
                  CALL wrf_dfi_write_initialized_state()
               END IF
   
            CASE DEFAULT
               wrf_err_message = 'Unrecognized DFI_OPT in namelist'
               CALL wrf_error_fatal3("<stdin>",462,&
TRIM(wrf_err_message))
   
         END SELECT
   
      END IF

   END SUBROUTINE wrf_dfi

   SUBROUTINE set_derived_rconfigs

      IMPLICIT NONE

      INTEGER :: i

      IF ( model_config_rec % dfi_opt .EQ. DFI_NODFI ) THEN
        DO i = 1, model_config_rec % max_dom
           model_config_rec % mp_physics_dfi(i) = -1
        ENDDO
      ELSE
        DO i = 1, model_config_rec % max_dom
           model_config_rec % mp_physics_dfi(i) = model_config_rec % mp_physics(i)
        ENDDO
      END IF

      IF ( model_config_rec % dfi_opt .EQ. DFI_NODFI ) THEN
        DO i = 1, model_config_rec % max_dom
           model_config_rec % bl_pbl_physics_dfi(i) = -1
        ENDDO
      ELSE
        DO i = 1, model_config_rec % max_dom
           model_config_rec % bl_pbl_physics_dfi(i) = model_config_rec % bl_pbl_physics(i)
        ENDDO
      END IF

   END SUBROUTINE set_derived_rconfigs

   RECURSIVE SUBROUTINE alloc_doms_for_dfi ( grid )

      TYPE (domain) , pointer :: grid
      TYPE (domain) , pointer :: new_nest_loc
      TYPE (grid_config_rec_type) :: parent_config_flags
      INTEGER :: nestid_loc , kid_loc
         DO WHILE ( nests_to_open( grid , nestid_loc , kid_loc ) )
            CALL alloc_and_configure_domain ( domain_id  = nestid_loc   , &
                                              grid       = new_nest_loc , &
                                              parent     = grid         , &
                                              kid        = kid_loc        )
         
print *,'for parent domain id #',grid%id,', found child domain #',nestid_loc

            new_nest_loc%dfi_opt = head_grid%dfi_opt
            new_nest_loc%dfi_stage = DFI_SETUP

            CALL Setup_Timekeeping (new_nest_loc)

            CALL model_to_grid_config_rec ( grid%id , model_config_rec , parent_config_flags )
            CALL med_nest_initial ( grid , new_nest_loc , config_flags )

            CALL alloc_doms_for_dfi ( new_nest_loc )
   
         END DO
   
   END SUBROUTINE alloc_doms_for_dfi

END MODULE module_wrf_top

```

## Python Translation

### `





---

# `module_driver_constants`

```{FORTRAN}
!WRF:DRIVER_LAYER:CONSTANTS
!
!  This MODULE contains all of the constants used in the model.  These
!  are separated by usage within the code.

MODULE module_driver_constants

   !  0. The following tells the rest of the model what data ordering we are
   !     using

   INTEGER , PARAMETER :: DATA_ORDER_XYZ = 1
   INTEGER , PARAMETER :: DATA_ORDER_YXZ = 2
   INTEGER , PARAMETER :: DATA_ORDER_ZXY = 3
   INTEGER , PARAMETER :: DATA_ORDER_ZYX = 4
   INTEGER , PARAMETER :: DATA_ORDER_XZY = 5
   INTEGER , PARAMETER :: DATA_ORDER_YZX = 6
   INTEGER , PARAMETER :: DATA_ORDER_XY = DATA_ORDER_XYZ
   INTEGER , PARAMETER :: DATA_ORDER_YX = DATA_ORDER_YXZ
   
```

In [44]:
def module_driver_constants_0():
    DATA_ORDER_XYZ = 1
    DATA_ORDER_YXZ = 2
    DATA_ORDER_ZXY = 3
    DATA_ORDER_ZYX = 4
    DATA_ORDER_XZY = 5
    DATA_ORDER_YZX = 6
    DATA_ORDER_XY = DATA_ORDER_XYZ
    DATA_ORDER_YX = DATA_ORDER_YXZ

```{FORTRAN}
    #include "model_data_order.inc"

       !  1. Following are constants for use in defining maximal values for array
       !     definitions.  
       !

       !  The maximum number of levels in the model is how deeply the domains may
       !  be nested.

       INTEGER , PARAMETER :: max_levels      =  20

       !  The maximum number of nests that can depend on a single parent and other way round

       INTEGER , PARAMETER :: max_nests        =  20

       !  The maximum number of parents that a nest can have (simplified assumption -> one only)

       INTEGER , PARAMETER :: max_parents      =  1

       !  The maximum number of domains is how many grids the model will be running.

       INTEGER , PARAMETER :: max_domains     =   ( MAX_DOMAINS_F - 1 ) / 2 + 1

       !  The maximum number of nest move specifications allowed in a namelist

       INTEGER , PARAMETER :: max_moves       =   50

       !  The maximum number of eta levels
       !DJW 140701 Increased from 501 to 1001 since I can imagine using more than
       !501 total vertical levels across multiple nested domains. Now that the
       !code is modified to allow specification of all domains eta_levels using a
       !array of length max_eta, this will need to be larger.  I'll also add a check
       !in module_initialize_real to ensure we don't exceed this value.

       INTEGER , PARAMETER :: max_eta         =   1001

       !  The maximum number of ocean levels in the 3d U Miami ocean.

       INTEGER , PARAMETER :: max_ocean       =   501

       !  The maximum number of pressure levels to interpolate to, for diagnostics

       INTEGER , PARAMETER :: max_plevs       =   100

       !  The maximum number of height levels to interpolate to, for diagnostics

       INTEGER , PARAMETER :: max_zlevs       =   100

       !  The maximum number of trackchem

       INTEGER , PARAMETER :: max_trackchem   =   100

       !  The maximum number of outer iterations (for DA minimisation)

       INTEGER , PARAMETER :: max_outer_iterations = 100

       !  The maximum number of instruments (for radiance DA)

       INTEGER , PARAMETER :: max_instruments =   30

       !  The maximum number of obs indexes (for conventional DA obs) 

       INTEGER , PARAMETER :: num_ob_indexes  =   28


       !  The maximum number of bogus storms

       INTEGER , PARAMETER :: max_bogus =  5

       !  The maximum number of fields that can be sent or received in coupled mode

       INTEGER , PARAMETER :: max_cplfld = 20

       !  The maximum number of domains used by the external model with which wrf is communicating in coupled mode

       INTEGER , PARAMETER :: max_extdomains = 5
```

In [46]:
def module_driver_constants_1():
    max_levels = 20
    max_nests = 20
    max_parents = 1
    max_domains = (MAX_DOMAINS_F - 1) / 2 + 1
    max_moves = 50
    max_eta = 1001
    max_ocean = 501
    max_plevs = 100
    max_zlevs = 100
    max_trackchem = 100
    max_outer_iterations = 100
    max_instruments = 30
    num_ob_indexes = 28
    max_bogus = 5
    max_cplfld = 20
    max_extdomains = 5

```{FORTRAN}
       !  2. Following related to driver level data structures for DM_PARALLEL communications

    #ifdef DM_PARALLEL
       INTEGER , PARAMETER :: max_comms       =   1024
    #else
       INTEGER , PARAMETER :: max_comms       =   1
    #endif
```

In [47]:
def module_driver_constants_2():
    try:
        DM_PARALLEL
    except:
        max_comms = 1024
    else:
        max_comms = 1

```{FORTRAN}
   !  3. Following is information related to the file I/O.

   !  These are the bounds of the available FORTRAN logical unit numbers for the file I/O.
   !  Only logical unit numbers within these bounds will be chosen for I/O unit numbers.

   INTEGER , PARAMETER :: min_file_unit = 10
   INTEGER , PARAMETER :: max_file_unit = 99
```

In [48]:
def module_driver_constants_3():
    min_file_unit = 10
    max_file_unit = 99

```{FORTRAN}
       !  4. Unfortunately, the following definition is needed here (rather
       !     than the more logical place in share/module_model_constants.F)
       !     for the namelist reads in frame/module_configure.F, and for some
       !     conversions in share/set_timekeeping.F
       !     Actually, using it here will mean that we don't need to set it
       !     in share/module_model_constants.F, since this file will be
       !     included (USEd) in:
       !        frame/module_configure.F
       !     which will be USEd in:
       !        share/module_bc.F
       !     which will be USEd in:
       !        phys/module_radiation_driver.F
       !     which is the other important place for it to be, and where
       !     it is passed as a subroutine parameter to any physics subroutine.
       !
       !     P2SI is the number of SI seconds in an planetary solar day
       !     divided by the number of SI seconds in an earth solar day
    #if defined MARS
       !     For Mars, P2SI = 88775.2/86400.
       REAL , PARAMETER :: P2SI = 1.0274907
    #elif defined TITAN
       !     For Titan, P2SI = 1378080.0/86400.
       REAL , PARAMETER :: P2SI = 15.95
    #else
       !     Default for Earth
       REAL , PARAMETER :: P2SI = 1.0
    #endif
     CONTAINS
       SUBROUTINE init_module_driver_constants
       END SUBROUTINE init_module_driver_constants
     END MODULE module_driver_constants
```

In [50]:
def module_driver_constants_4():
    P2SI = 1.0
    

```{FORTRAN}
    ! routines that external packages can call to get at WRF stuff that isn't available
    ! through argument lists; since they are external we don't want them using WRF 
    ! modules unnecessarily (complicates the build even more)
     SUBROUTINE inquire_of_wrf_data_order_xyz( data_order )
       USE module_driver_constants, ONLY : DATA_ORDER_XYZ
       IMPLICIT NONE
       INTEGER, INTENT(OUT) :: data_order
       data_order = DATA_ORDER_XYZ
     END SUBROUTINE inquire_of_wrf_data_order_xyz

     SUBROUTINE inquire_of_wrf_data_order_xzy( data_order )
       USE module_driver_constants, ONLY : DATA_ORDER_XZY
       IMPLICIT NONE
       INTEGER, INTENT(OUT) :: data_order
       data_order = DATA_ORDER_XZY
     END SUBROUTINE inquire_of_wrf_data_order_xzy

     SUBROUTINE inquire_of_wrf_iwordsize( iwordsz )
       IMPLICIT NONE
       INTEGER, INTENT(OUT) :: iwordsz
       iwordsz = IWORDSIZE
     END SUBROUTINE inquire_of_wrf_iwordsize

     SUBROUTINE inquire_of_wrf_rwordsize( rwordsz )
       IMPLICIT NONE
       INTEGER, INTENT(OUT) :: rwordsz
       rwordsz = RWORDSIZE
     END SUBROUTINE inquire_of_wrf_rwordsize

```

In [53]:
def module_driver_constants():
    module_driver_constants_0()
    module_driver_constants_1()
    module_driver_constants_2()
    module_driver_constants_3()
    module_driver_constants_4()

# module_driver_constants()
# NEED MAX_DOMAINS_F

---

### `module_machine`

```{FORTRAN}
MODULE module_machine

   USE module_driver_constants

   INTEGER, PARAMETER :: TILE_NONE = 0, TILE_X = 1, TILE_Y = 2, TILE_XY = 3

   CONTAINS
```

#### `rlocproc`
```{FORTRAN}
   RECURSIVE SUBROUTINE rlocproc(p,maxi,nproc,ml,mr,ret)
   IMPLICIT NONE
   INTEGER, INTENT(IN)  :: p, maxi, nproc, ml, mr
   INTEGER, INTENT(OUT) :: ret
   INTEGER              :: width, rem, ret2, bl, br, mid, adjust, &
                           p_r, maxi_r, nproc_r, zero
   adjust = 0
   rem = mod( maxi, nproc )
   width = maxi / nproc
   mid = maxi / 2
   
      IF ( rem>0 .AND. (((mod(rem,2).EQ.0).OR.(rem.GT.2)).OR.(p.LE.mid))) THEN
     width = width + 1
    END IF
    IF ( p.LE.mid .AND. mod(rem,2).NE.0 ) THEN
     adjust = adjust + 1
    END IF
    bl = max(width,ml) ;
    br = max(width,mr) ;
    IF      (p<bl) THEN
     ret = 0
    ELSE IF (p>maxi-br-1) THEN
     ret = nproc-1
    ELSE
     p_r = p - bl
     maxi_r = maxi-bl-br+adjust
     nproc_r = max(nproc-2,1)
     zero = 0
     CALL rlocproc( p_r, maxi_r, nproc_r, zero, zero, ret2 )  
     ret = ret2 + 1
    END IF
    RETURN
    END SUBROUTINE rlocproc
```


In [40]:
TILE_NONE = 0
TILE_X = 1
TILE_Y = 2
TILE_XY = 3

def rlocproc(p, maxi, nproc, ml, mr, ret):

    adjust = 0
    rem = np.remainder(maxi, nproc)
    width = maxi / nproc
    mid = maxi / 2
    
    if ((rem > 0) and (((np.remainder(rem, 2) == 0) or (rem > 2)) or (p < mid))):
        width = width + 1
        
    if ((p <= mid) and (np.remainder(rem, 2) != 0)):
        adjust = adjust + 1
    
    bl = max(width, ml)
    br = max(width, mr)
    
    if p < bl:  # goes if p < 3 or (30 / 100)
        ret = 0
    elif p > (maxi - br - 1): # goes if p > 26 (30 - 3 - 1 = p > 26)
        ret = nproc - 1
    else: # goes if 26 > p > 3
        p_r = p - bl
        maxi_r = maxi - bl - br + adjust
        nproc_r = max(nproc-2, 1)
        zero = 0
        
    rlocproc(p_r, maxi_r, nproc_r, zero, zero, ret2)
    ret = ret2 + 1
    return ret

In [41]:
# rlocproc(20, 30, 100, 3, 3, 4)
# NEED ret2

```{FORTRAN}
   INTEGER FUNCTION locproc( i, m, numpart )
   implicit none
   integer, intent(in) :: i, m, numpart 
   integer             :: retval, ii, im, inumpart, zero
   ii = i
   im = m
   inumpart = numpart
   zero = 0
   CALL rlocproc( ii, im, inumpart, zero, zero, retval )
   locproc = retval
   RETURN
   END FUNCTION locproc
```

In [43]:
def locproc(i, m, numpart):
    ii = i
    im = m
    inpart = numpart
    zero = 0
    rlocproc(ii, im, inumpart, zero, zero, retval)

```{FORTRAN}
   SUBROUTINE patchmap( res, y, x, py, px )
   implicit none
   INTEGER, INTENT(IN)                    :: y, x, py, px
   INTEGER, DIMENSION(x,y), INTENT(OUT)   :: res
   INTEGER                                :: i, j, p_min, p_maj
   DO j = 0,y-1
     p_maj = locproc( j, y, py )
     DO i = 0,x-1
       p_min = locproc( i, x, px )
       res(i+1,j+1) = p_min + px*p_maj
     END DO
   END DO
   RETURN
   END SUBROUTINE patchmap
```



In [ ]:
def patchmap(res, y, x, py, px):
    for j in np.arange(0, y - 1):
        p_maj = locproc(k, y, py)
        for i np.arange(0, x - 1):
            p_min = locproc(i, x, px)
            res(i + 1, j + 1) = p_min + px * p_maj
    return res

```{FORTRAN}
   SUBROUTINE region_bounds( region_start, region_end, &
                             num_p, p,                 &
                             patch_start, patch_end )

   IMPLICIT NONE
   INTEGER, INTENT(IN)                    :: region_start, region_end, num_p, p
   INTEGER, INTENT(OUT)                   :: patch_start, patch_end
   INTEGER                                :: offset, i
   patch_end = -999999999
   patch_start = 999999999
   offset = region_start
   do i = 0, region_end - offset
     if ( locproc( i, region_end-region_start+1, num_p ) == p ) then
       patch_end = max(patch_end,i)
       patch_start = min(patch_start,i)
     endif
   enddo
   patch_start = patch_start + offset
   patch_end   = patch_end   + offset
   RETURN
   END SUBROUTINE region_bounds
```

In [ ]:
def region_bounds(region_start, region_end, num_p, p, patch_start, patch_end):
    patch_end = -999999999
    patch_start = 999999999
    offset = region_start
    for i in np.arange(0, region_end - offset):
        if (locproc(i, region_end - region_start + 1, num_p) == p):
            patch_end = max(patch_end, i)
            patch_start = min(patch_start, i)
    patch_start = patch_start + offset
    patch_end   = patch_end   + offset
    
    return patch_start, patch_end

```{FORTRAN}
   SUBROUTINE least_aspect( nparts, minparts_y, minparts_x, nparts_y, nparts_x )
   IMPLICIT NONE

   INTEGER, INTENT(IN)           :: nparts,                &
                                    minparts_y, minparts_x

   INTEGER, INTENT(OUT)          :: nparts_y, nparts_x

   INTEGER                       :: x, y, mini
   mini = 2*nparts
   nparts_y = 1
   nparts_x = nparts
   DO y = 1, nparts
      IF ( mod( nparts, y ) .eq. 0 ) THEN
         x = nparts / y
         IF (       abs( y-x ) .LT. mini       &
              .AND. y .GE. minparts_y                &
              .AND. x .GE. minparts_x    ) THEN
            mini = abs( y-x )
            nparts_y = y
            nparts_x = x
         END IF
      END IF
   END DO
   END SUBROUTINE least_aspect
```

In [ ]:
def least_aspect(nparts, minparts_y, minparts_x, nparts_y, nparts_x):
    mini = 2 * nparts
    nparts_y = 1
    nparts_x = nparts
    for y in np.arange(1, nparts):
        if np.remainder(nparts, y) == 0:
            x = nparts / y
            if (np.abs(y - x) < mini) and (y >= minparts_y) and x >= minparts_x:
                mini = np.abs(y - x)
                nparts_y = y
                nparts_x = x
    
    return nparts_y, nparts_x

```{FORTRAN}
       SUBROUTINE init_module_machine
         RETURN
       END SUBROUTINE init_module_machine

    END MODULE module_machine

    SUBROUTINE wrf_sizeof_integer( retval )
      IMPLICIT NONE
      INTEGER retval

      retval = 4
      RETURN
    END SUBROUTINE wrf_sizeof_integer

    SUBROUTINE wrf_sizeof_real( retval )
      IMPLICIT NONE
      INTEGER retval

      retval = 4
      RETURN
    END SUBROUTINE wrf_sizeof_real

    SUBROUTINE wrf_sizeof_doubleprecision( retval )
      IMPLICIT NONE
      INTEGER retval

      retval = 8
      RETURN
    END SUBROUTINE wrf_sizeof_doubleprecision

    SUBROUTINE wrf_sizeof_logical( retval )
      IMPLICIT NONE
      INTEGER retval

      retval = 4
      RETURN
    END SUBROUTINE wrf_sizeof_logical
```

In [56]:
def init_module_machine():
    return

def wrf_sizeof_integer(retval):
    retvar = 4
    
def wrf_sizeof_real(retval):
    retval = 4
    
def wrf_sizeof_doubleprecision(retval):
    retval = 8
    
def wrf_sizeof_logical(retval):
    retval = 4

### 

```{FORTRAN}
                   IMPLICIT NONE

                   REAL    :: time
                   INTEGER :: loop , &
                              levels_to_process
                   TYPE (domain) , POINTER :: keep_grid, grid_ptr, null_domain
                   TYPE (domain) , pointer :: parent_grid, new_nest
                   LOGICAL                                :: a_nest_was_opened
                   TYPE (grid_config_rec_type), SAVE :: config_flags
                   INTEGER        :: kid, nestid
                   INTEGER                 :: number_at_same_level
                   INTEGER                 :: time_step_begin_restart
                   INTEGER :: max_dom , domain_id , fid , oid , idum1 , idum2 , ierr
                   INTEGER :: debug_level
                   LOGICAL :: input_from_file
                   INTEGER                 :: nbytes
                   INTEGER, PARAMETER      :: configbuflen = 4* 65536
                   INTEGER                 :: configbuf( configbuflen )
                   LOGICAL , EXTERNAL      :: wrf_dm_on_monitor
                   CHARACTER (LEN=256)     :: rstname
                   CHARACTER (LEN=80)      :: message
                   CHARACTER (LEN=256) , PRIVATE :: a_message

                   INTERFACE 
                     SUBROUTINE Setup_Timekeeping( grid )
                      USE module_domain
                      TYPE(domain), POINTER :: grid
                     END SUBROUTINE Setup_Timekeeping

                     SUBROUTINE wrf_dfi_write_initialized_state( )
                     END SUBROUTINE wrf_dfi_write_initialized_state

                     SUBROUTINE wrf_dfi_startfwd_init( )
                     END SUBROUTINE wrf_dfi_startfwd_init

                     SUBROUTINE wrf_dfi_startbck_init( )
                     END SUBROUTINE wrf_dfi_startbck_init

                     SUBROUTINE wrf_dfi_bck_init( )
                     END SUBROUTINE wrf_dfi_bck_init

                     SUBROUTINE wrf_dfi_fwd_init( )
                     END SUBROUTINE wrf_dfi_fwd_init

                     SUBROUTINE wrf_dfi_fst_init( )
                     END SUBROUTINE wrf_dfi_fst_init

                     SUBROUTINE wrf_dfi_array_reset ( )
                     END SUBROUTINE wrf_dfi_array_reset

                     SUBROUTINE med_nest_initial ( parent , grid , config_flags )
                       USE module_domain
                       USE module_configure
                       TYPE (domain), POINTER ::  grid , parent
                       TYPE (grid_config_rec_type) config_flags
                     END SUBROUTINE med_nest_initial

                   END INTERFACE

                CONTAINS
                ```

                ### `wrf_init`

                ```{FORTRAN}
                   SUBROUTINE wrf_init( no_init1 )

                     LOGICAL, OPTIONAL, INTENT(IN) :: no_init1
                     INTEGER i, myproc, nproc, hostid, loccomm, ierr, buddcounter, mydevice, save_comm
                     INTEGER, ALLOCATABLE :: hostids(:), budds(:)
                     CHARACTER*512 hostname
                     CHARACTER*512 mminlu_loc
                   CHARACTER (LEN=10) :: release_version = 'V3.8.1    '

                   program_name = "WRF " // TRIM(release_version) // " MODEL"

                   CALL init_modules(1)
                   IF ( .NOT. PRESENT( no_init1 ) ) THEN

                     CALL WRFU_Initialize( defaultCalKind=WRFU_CAL_GREGORIAN )
                   ENDIF

                   CALL init_modules(2)

                   CALL wrf_get_dm_communicator( save_comm )
                   CALL wrf_set_dm_communicator( mpi_comm_allcompute )
                   IF ( wrf_dm_on_monitor() ) THEN
                     CALL initial_config
                   ENDIF
                   CALL get_config_as_buffer( configbuf, configbuflen, nbytes )
                   CALL wrf_dm_bcast_bytes( configbuf, nbytes )
                   CALL set_config_as_buffer( configbuf, configbuflen )
                   CALL wrf_dm_initialize
                   CALL wrf_set_dm_communicator( save_comm )

                   CALL set_derived_rconfigs
                   CALL check_nml_consistency
                   CALL set_physics_rconfigs

                   CALL nl_get_debug_level ( 1, debug_level )
                   CALL set_wrf_debug_level ( debug_level )

                   NULLIFY( null_domain )

                   CALL nl_get_max_dom( 1, max_dom )
                   IF ( max_dom > 1 ) THEN
                   END IF

                   CALL       wrf_message ( program_name )
                   CALL       wrf_debug ( 100 , 'wrf: calling alloc_and_configure_domain ' )
                   CALL alloc_and_configure_domain ( domain_id  = 1 ,                  &
                                               active_this_task = domain_active_this_task(1), &
                                                     grid       = head_grid ,          &
                                                     parent     = null_domain ,        &
                                                     kid        = -1                   )

                   CALL       wrf_debug ( 100 , 'wrf: calling model_to_grid_config_rec ' )
                   CALL model_to_grid_config_rec ( head_grid%id , model_config_rec , config_flags )
                   CALL       wrf_debug ( 100 , 'wrf: calling set_scalar_indices_from_config ' )
                   CALL set_scalar_indices_from_config ( head_grid%id , idum1, idum2 )
                   CALL       wrf_debug ( 100 , 'wrf: calling init_wrfio' )
                   CALL init_wrfio

                   CALL wrf_get_dm_communicator( save_comm )
                   CALL wrf_set_dm_communicator( mpi_comm_allcompute )
                   CALL get_config_as_buffer( configbuf, configbuflen, nbytes )
                   CALL wrf_dm_bcast_bytes( configbuf, nbytes )
                   CALL set_config_as_buffer( configbuf, configbuflen )
                   CALL wrf_set_dm_communicator( save_comm )

                   IF ( head_grid%dfi_opt .NE. DFI_NODFI ) head_grid%dfi_stage = DFI_SETUP

                   CALL Setup_Timekeeping (head_grid)

                 IF ( domain_active_this_task(1) ) THEN
                   CALL med_initialdata_input( head_grid , config_flags )

                   IF ( config_flags%write_restart_at_0h ) THEN
                      CALL med_restart_out ( head_grid, config_flags )

                      CALL wrf_debug ( 0 , ' 0 h restart only wrf: SUCCESS COMPLETE WRF' )

                      CALL wrf_finalize( )
                   END IF
                  ENDIF  

                   head_grid%start_subtime = domain_get_start_time ( head_grid )
                   head_grid%stop_subtime = domain_get_stop_time ( head_grid )

                 IF ( domain_active_this_task(1) ) THEN

                   IF ( head_grid%dfi_opt .NE. DFI_NODFI ) THEN
                      CALL alloc_doms_for_dfi ( head_grid )
                   END IF

                   IF (coupler_on) CALL cpl_defdomain( head_grid ) 
                  ENDIF  

                   END SUBROUTINE wrf_init
```

## `wrf_run`

```{FORTRAN}
           SUBROUTINE wrf_run( )

           CALL       wrf_debug ( 100 , 'wrf: calling integrate' )
           CALL integrate ( head_grid )
           CALL       wrf_debug ( 100 , 'wrf: back from integrate' )

           END SUBROUTINE wrf_run
```

## `wrf_finalize`

```{FORTRAN}
           SUBROUTINE wrf_finalize( no_shutdown )

             LOGICAL, OPTIONAL, INTENT(IN) :: no_shutdown

           CALL med_shutdown_io ( head_grid , config_flags )
           CALL       wrf_debug ( 100 , 'wrf: back from med_shutdown_io' )

           CALL       wrf_debug (   0 , 'wrf: SUCCESS COMPLETE WRF' )

           IF ( .NOT. PRESENT( no_shutdown ) ) THEN

              IF (coupler_on) THEN 
                 CALL cpl_finalize() 
              ELSE
                 CALL WRFU_Finalize
                 CALL wrf_shutdown
              ENDIF
           ENDIF

           END SUBROUTINE wrf_finalize
```

## `wrf_dfi`

```{FORTRAN}
           SUBROUTINE wrf_dfi()

              IMPLICIT NONE

              IF ( config_flags%dfi_opt .NE. DFI_NODFI ) THEN

                 SELECT CASE ( config_flags%dfi_opt ) 

                    CASE (DFI_DFL)
                       wrf_err_message = 'Initializing with DFL'
                       CALL wrf_message(TRIM(wrf_err_message))

                       wrf_err_message = '   Filtering forward in time'
                       CALL wrf_message(TRIM(wrf_err_message))

                       CALL wrf_dfi_fwd_init()
                       CALL wrf_run()

                       CALL wrf_dfi_array_reset()

                       CALL wrf_dfi_fst_init()

                       IF ( config_flags%dfi_write_filtered_input ) THEN
                          CALL wrf_dfi_write_initialized_state()
                       END IF

                    CASE (DFI_DDFI)
                       wrf_err_message = 'Initializing with DDFI'
                       CALL wrf_message(TRIM(wrf_err_message))

                       wrf_err_message = '   Integrating backward in time'
                       CALL wrf_message(TRIM(wrf_err_message))

                       CALL wrf_dfi_bck_init()
                       CALL wrf_run()

                       wrf_err_message = '   Filtering forward in time'
                       CALL wrf_message(TRIM(wrf_err_message))

                       CALL wrf_dfi_fwd_init()
                       CALL wrf_run()

                       CALL wrf_dfi_array_reset()

                       CALL wrf_dfi_fst_init()

                       IF ( config_flags%dfi_write_filtered_input ) THEN
                          CALL wrf_dfi_write_initialized_state()
                       END IF

                    CASE (DFI_TDFI)
                       wrf_err_message = 'Initializing with TDFI'
                       CALL wrf_message(TRIM(wrf_err_message))

                       wrf_err_message = '   Integrating backward in time'
                       CALL wrf_message(TRIM(wrf_err_message))

                       CALL wrf_dfi_bck_init()
                       CALL wrf_run()

                       CALL wrf_dfi_array_reset()

                       wrf_err_message = '   Filtering forward in time'
                       CALL wrf_message(TRIM(wrf_err_message))

                       CALL wrf_dfi_fwd_init()
                       CALL wrf_run()

                       CALL wrf_dfi_array_reset()

                       CALL wrf_dfi_fst_init()

                       IF ( config_flags%dfi_write_filtered_input ) THEN
                          CALL wrf_dfi_write_initialized_state()
                       END IF

                    CASE DEFAULT
                       wrf_err_message = 'Unrecognized DFI_OPT in namelist'
                       CALL wrf_error_fatal3("<stdin>",462,&
        TRIM(wrf_err_message))

                 END SELECT

              END IF

           END SUBROUTINE wrf_dfi
```

```{FORTRAN}
           SUBROUTINE set_derived_rconfigs

              IMPLICIT NONE

              INTEGER :: i

              IF ( model_config_rec % dfi_opt .EQ. DFI_NODFI ) THEN
                DO i = 1, model_config_rec % max_dom
                   model_config_rec % mp_physics_dfi(i) = -1
                ENDDO
              ELSE
                DO i = 1, model_config_rec % max_dom
                   model_config_rec % mp_physics_dfi(i) = model_config_rec % mp_physics(i)
                ENDDO
              END IF

              IF ( model_config_rec % dfi_opt .EQ. DFI_NODFI ) THEN
                DO i = 1, model_config_rec % max_dom
                   model_config_rec % bl_pbl_physics_dfi(i) = -1
                ENDDO
              ELSE
                DO i = 1, model_config_rec % max_dom
                   model_config_rec % bl_pbl_physics_dfi(i) = model_config_rec % bl_pbl_physics(i)
                ENDDO
              END IF

           END SUBROUTINE set_derived_rconfigs

           RECURSIVE SUBROUTINE alloc_doms_for_dfi ( grid )

              TYPE (domain) , pointer :: grid

              TYPE (domain) , pointer :: new_nest_loc
              TYPE (grid_config_rec_type) :: parent_config_flags
              INTEGER :: nestid_loc , kid_loc

                 DO WHILE ( nests_to_open( grid , nestid_loc , kid_loc ) )

                    CALL alloc_and_configure_domain ( domain_id  = nestid_loc   , &
                                                      grid       = new_nest_loc , &
                                                      parent     = grid         , &
                                                      kid        = kid_loc        )

        print *,'for parent domain id #',grid%id,', found child domain #',nestid_loc

                    new_nest_loc%dfi_opt = head_grid%dfi_opt
                    new_nest_loc%dfi_stage = DFI_SETUP

                    CALL Setup_Timekeeping (new_nest_loc)

                    CALL model_to_grid_config_rec ( grid%id , model_config_rec , parent_config_flags )
                    CALL med_nest_initial ( grid , new_nest_loc , config_flags )

                    CALL alloc_doms_for_dfi ( new_nest_loc )

                 END DO

           END SUBROUTINE alloc_doms_for_dfi

        END MODULE module_wrf_top
```